In [1]:
import langchain
from langchain_ollama import ChatOllama


In [8]:
chat = ChatOllama(
    model="llama3.2",
    temperature=0.0,
)
print(chat)

model='llama3.2' temperature=0.0


In [9]:
from langchain.prompts import ChatPromptTemplate

template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""
prompt_template = ChatPromptTemplate.from_template(template_string)

print(prompt_template.messages[0],"\n")
print(prompt_template.messages[0].prompt,"\n")
print(prompt_template.messages[0].prompt.input_variables)

prompt=PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n') additional_kwargs={} 

input_variables=['style', 'text'] input_types={} partial_variables={} template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n' 

['style', 'text']


In [10]:
customer_style = """American English \
in a calm and respectful tone
"""
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""
customer_message = prompt_template.format_messages(style=customer_style, text=customer_email)

In [11]:
print(type(customer_message))
print(type(customer_message[0]))
print(customer_message)

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>
[HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n", additional_kwargs={}, response_metadata={})]


In [12]:
messages = [
    {"role": "system", "content": "You are a helpful assistant! Your name is Bob."},
    {"role": "user", "content": "What is your name?"}
]
response = chat.invoke(messages)
print(response)

content="My name is Bob, nice to meet you! I'm here to help answer any questions or provide information on a wide range of topics. How can I assist you today?" additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2024-12-20T05:51:52.718193Z', 'done': True, 'done_reason': 'stop', 'total_duration': 23692475500, 'load_duration': 9504105600, 'prompt_eval_count': 41, 'prompt_eval_duration': 7494000000, 'eval_count': 36, 'eval_duration': 4333000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-03af7381-9c49-4e30-94de-059aa7c37270-0' usage_metadata={'input_tokens': 41, 'output_tokens': 36, 'total_tokens': 77}


In [4]:
print(langchain.__version__)

0.3.10


In [13]:
customer_response = chat.invoke(customer_message)
print(customer_response)

content='Here\'s a rewritten version of the text in an American English style, maintaining a calm and respectful tone:\n\n"I\'m really frustrated that my blender lid came loose and spilled smoothie all over my kitchen walls. To make matters worse, the warranty doesn\'t cover the cost of cleaning up the mess. I could use some assistance right now."\n\nI made the following changes to adapt the text to American English:\n\n- Changed "Arrr" to a more neutral phrase ("I\'m really frustrated")\n- Replaced "me" with "my"\n- Changed "matey" to a more formal expression ("I could use some assistance")\n- Used more straightforward language throughout' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2024-12-20T05:52:54.8612784Z', 'done': True, 'done_reason': 'stop', 'total_duration': 23247143200, 'load_duration': 2186419400, 'prompt_eval_count': 107, 'prompt_eval_duration': 5067000000, 'eval_count': 132, 'eval_duration': 15991000000, 'message': Message(role='assistant',

## Output Parsing

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

response_schemas = [ResponseSchema(name="gift", description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown."),
                    ResponseSchema(name="delivery_days", description="How many days\
                                   did it take for the item to be delivered? If this \
                                   information is not found, output -1"),
                    ResponseSchema(name="price_value", description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")
                    ]
output_parser = StructuredOutputParser(response_schemas=response_schemas)
format_instructions = output_parser.get_format_instructions()

In [9]:
print(output_parser)
print(type(output_parser))

response_schemas=[ResponseSchema(name='gift', description='Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.', type='string'), ResponseSchema(name='delivery_days', description='How many days                                   did it take for the item to be delivered? If this                                    information is not found, output -1', type='string'), ResponseSchema(name='price_value', description='Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.', type='string')]
<class 'langchain.output_parsers.structured.StructuredOutputParser'>


In [10]:
print(format_instructions)
print(type(format_instructions))

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                   did it take for the item to be delivered? If this                                    information is not found, output -1
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```
<class 'str'>


In [11]:

customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""
template_string = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}
{format_instructions}
"""

prompt_template = ChatPromptTemplate.from_template(template_string)
message = prompt_template.format_messages(text=customer_review, format_instructions=format_instructions)
print(message)

[HumanMessage(content='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\ntext: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife\'s anniversary present. I think my wife liked it so much she was speechless. So far I\'ve been the only one using it, and I\'ve been using it every other morning to clear the leaves on our lawn. It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features.\n\nThe output should be a markdown code snippet formatted in the following schema, including the le

In [14]:
from langchain_ollama import ChatOllama
chat = ChatOllama(model="llama3.2", temperature=0.0)

response = chat.invoke(message)
print(response.content)

```json
{
    "gift": "False",
    "delivery_days": "2",
    "price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [15]:
response_dict = output_parser.parse(response.content)
print(response_dict)

{'gift': 'False', 'delivery_days': '2', 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}
